In [1]:
from policyengine_us import Microsimulation

sim = Microsimulation(dataset = "hf://policyengine/test/sparse_cd_stacked_2023.h5")
cd_geoids = sim.calculate("congressional_district_geoid").values
correct_state_fips = cd_geoids // 100
sim.set_input("state_fips", 2023, correct_state_fips)


/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from policyengine_us import Microsimulation
from policyengine_us.variables.input.geography import StateName

sim = Microsimulation(dataset = "hf://policyengine/test/sparse_cd_stacked_2023.h5")
YEAR = 2023

STATE_FIPS_TO_NAME = {
    1: StateName.AL, 2: StateName.AK, 4: StateName.AZ, 5: StateName.AR, 6: StateName.CA,
    8: StateName.CO, 9: StateName.CT, 10: StateName.DE, 11: StateName.DC,
    12: StateName.FL, 13: StateName.GA, 15: StateName.HI, 16: StateName.ID, 17: StateName.IL,
    18: StateName.IN, 19: StateName.IA, 20: StateName.KS, 21: StateName.KY, 22: StateName.LA,
    23: StateName.ME, 24: StateName.MD, 25: StateName.MA, 26: StateName.MI,
    27: StateName.MN, 28: StateName.MS, 29: StateName.MO, 30: StateName.MT,
    31: StateName.NE, 32: StateName.NV, 33: StateName.NH, 34: StateName.NJ,
    35: StateName.NM, 36: StateName.NY, 37: StateName.NC, 38: StateName.ND,
    39: StateName.OH, 40: StateName.OK, 41: StateName.OR, 42: StateName.PA,
    44: StateName.RI, 45: StateName.SC, 46: StateName.SD, 47: StateName.TN,
    48: StateName.TX, 49: StateName.UT, 50: StateName.VT, 51: StateName.VA, 53: StateName.WA,
    54: StateName.WV, 55: StateName.WI, 56: StateName.WY
}


cd_geoids = sim.calculate("congressional_district_geoid").values
correct_state_fips = cd_geoids // 100
correct_state_names = pd.Series(correct_state_fips).map(STATE_FIPS_TO_NAME).values

sim.set_input("state_fips", YEAR, correct_state_fips)

# Delete any cached calculations to force recalculation
if "state_name" in sim.tax_benefit_system.variables:
    sim.delete_arrays("state_name", YEAR)
if "state_code" in sim.tax_benefit_system.variables:
    sim.delete_arrays("state_code", YEAR)



In [3]:
state_fips = sim.calculate("state_fips", map_to="household", period=2026)


In [4]:
congressional_district_geoid = sim.calculate("congressional_district_geoid", map_to="household", period=2026)


In [5]:
income_tax = sim.calculate("income_tax", map_to="household", period=2026)


In [6]:
state_name = sim.calculate("state_name", map_to="household", period=2026)


In [7]:
state_code = sim.calculate("state_code", map_to="household", period=2026)


In [8]:
in_nj = state_code == "NJ"

In [9]:
mean_fed_tax_in_nj = income_tax[in_nj].mean()

In [10]:
mean_fed_tax_in_nj

26613.23385910318

In [11]:
fed_tax_in_nj = income_tax[in_nj]
districtics_in_nj = congressional_district_geoid[in_nj]

unique_districts = np.unique(districtics_in_nj)
district_list = {}

for district in unique_districts:
    in_district = districtics_in_nj == district
    mean_tax = fed_tax_in_nj[in_district].mean()
    district_list[district] = mean_tax

district_list

{3401: 21626.254254479445,
 3402: 19496.141684997117,
 3403: 26277.74194296395,
 3404: 32628.926321682633,
 3405: 28071.03803417276,
 3406: 24837.961113839345,
 3407: 35728.95922826653,
 3408: 19402.57601023985,
 3409: 23163.47901356361,
 3410: 21838.69476117316,
 3411: 31695.259674954348,
 3412: 29165.225455496624}

In [12]:
from policyengine_core.reforms import Reform


In [13]:

reform = Reform.from_dict({
  "gov.irs.credits.estate.base": {
    "2026-01-01.2026-12-31": 6790000,
    "2027-01-01.2027-12-31": 6960000,
    "2028-01-01.2028-12-31": 7100000,
    "2029-01-01.2029-12-31": 7240000,
    "2030-01-01.2030-12-31": 7390000,
    "2031-01-01.2031-12-31": 7530000,
    "2032-01-01.2032-12-31": 7680000,
    "2033-01-01.2033-12-31": 7830000,
    "2034-01-01.2034-12-31": 7990000,
    "2035-01-01.2100-12-31": 8150000
  },
  "gov.irs.income.bracket.rates.2": {
    "2025-01-01.2100-12-31": 0.15
  },
  "gov.irs.income.bracket.rates.3": {
    "2025-01-01.2100-12-31": 0.25
  },
  "gov.irs.income.bracket.rates.4": {
    "2025-01-01.2100-12-31": 0.28
  },
  "gov.irs.income.bracket.rates.5": {
    "2025-01-01.2100-12-31": 0.33
  },
  "gov.irs.income.bracket.rates.7": {
    "2025-01-01.2100-12-31": 0.396
  },
  "gov.irs.deductions.qbi.max.rate": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.income.exemption.amount": {
    "2026-01-01.2026-12-31": 5300,
    "2027-01-01.2027-12-31": 5400,
    "2028-01-01.2028-12-31": 5500,
    "2029-01-01.2029-12-31": 5650,
    "2030-01-01.2030-12-31": 5750,
    "2031-01-01.2031-12-31": 5850,
    "2032-01-01.2032-12-31": 5950,
    "2033-01-01.2033-12-31": 6100,
    "2034-01-01.2034-12-31": 6200,
    "2035-01-01.2100-12-31": 6350
  },
  "gov.irs.deductions.tip_income.cap": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.cdcc.phase_out.max": {
    "2026-01-01.2100-12-31": 0.35
  },
  "gov.irs.credits.cdcc.phase_out.min": {
    "2026-01-01.2100-12-31": 0.2
  },
  "gov.irs.deductions.qbi.max.w2_wages.rate": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.standard.amount.JOINT": {
    "2025-01-01.2025-12-31": 30000,
    "2026-01-01.2026-12-31": 16600,
    "2027-01-01.2027-12-31": 16900,
    "2028-01-01.2028-12-31": 17300,
    "2029-01-01.2029-12-31": 17600,
    "2030-01-01.2030-12-31": 18000,
    "2031-01-01.2031-12-31": 18300,
    "2032-01-01.2032-12-31": 18700,
    "2033-01-01.2033-12-31": 19000,
    "2034-01-01.2034-12-31": 19400,
    "2035-01-01.2100-12-31": 19800
  },
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2025-01-01.2025-12-31": 2000,
    "2026-01-01.2100-12-31": 1000
  },
  "gov.irs.deductions.auto_loan_interest.cap": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.standard.amount.SINGLE": {
    "2025-01-01.2025-12-31": 15000,
    "2026-01-01.2026-12-31": 8300,
    "2027-01-01.2027-12-31": 8450,
    "2028-01-01.2028-12-31": 8650,
    "2029-01-01.2029-12-31": 8800,
    "2030-01-01.2030-12-31": 9000,
    "2031-01-01.2031-12-31": 9150,
    "2032-01-01.2032-12-31": 9350,
    "2033-01-01.2033-12-31": 9500,
    "2034-01-01.2034-12-31": 9700,
    "2035-01-01.2100-12-31": 9900
  },
  "gov.irs.income.amt.exemption.amount.JOINT": {
    "2026-01-01.2026-12-31": 109800,
    "2027-01-01.2027-12-31": 112100,
    "2028-01-01.2028-12-31": 114400,
    "2029-01-01.2029-12-31": 116700,
    "2030-01-01.2030-12-31": 119000,
    "2031-01-01.2031-12-31": 121300,
    "2032-01-01.2032-12-31": 123700,
    "2033-01-01.2033-12-31": 126200,
    "2034-01-01.2034-12-31": 128700,
    "2035-01-01.2100-12-31": 131200
  },
  "gov.irs.income.bracket.thresholds.1.JOINT": {
    "2026-01-01.2026-12-31": 24300,
    "2027-01-01.2027-12-31": 24800,
    "2028-01-01.2028-12-31": 25300,
    "2029-01-01.2029-12-31": 25800,
    "2030-01-01.2030-12-31": 26300,
    "2031-01-01.2031-12-31": 26850,
    "2032-01-01.2032-12-31": 27350,
    "2033-01-01.2033-12-31": 27900,
    "2034-01-01.2034-12-31": 28450,
    "2035-01-01.2100-12-31": 29000
  },
  "gov.irs.income.bracket.thresholds.2.JOINT": {
    "2026-01-01.2026-12-31": 98600,
    "2027-01-01.2027-12-31": 100700,
    "2028-01-01.2028-12-31": 102800,
    "2029-01-01.2029-12-31": 104800,
    "2030-01-01.2030-12-31": 106900,
    "2031-01-01.2031-12-31": 109000,
    "2032-01-01.2032-12-31": 111100,
    "2033-01-01.2033-12-31": 113300,
    "2034-01-01.2034-12-31": 115600,
    "2035-01-01.2100-12-31": 117900
  },
  "gov.irs.income.bracket.thresholds.3.JOINT": {
    "2026-01-01.2026-12-31": 199000,
    "2027-01-01.2027-12-31": 203250,
    "2028-01-01.2028-12-31": 207350,
    "2029-01-01.2029-12-31": 211450,
    "2030-01-01.2030-12-31": 215600,
    "2031-01-01.2031-12-31": 219900,
    "2032-01-01.2032-12-31": 224250,
    "2033-01-01.2033-12-31": 228700,
    "2034-01-01.2034-12-31": 233200,
    "2035-01-01.2100-12-31": 237850
  },
  "gov.irs.income.bracket.thresholds.4.JOINT": {
    "2026-01-01.2026-12-31": 303250,
    "2027-01-01.2027-12-31": 309700,
    "2028-01-01.2028-12-31": 315950,
    "2029-01-01.2029-12-31": 322200,
    "2030-01-01.2030-12-31": 328550,
    "2031-01-01.2031-12-31": 335050,
    "2032-01-01.2032-12-31": 341700,
    "2033-01-01.2033-12-31": 348450,
    "2034-01-01.2034-12-31": 355400,
    "2035-01-01.2100-12-31": 362450
  },
  "gov.irs.income.bracket.thresholds.5.JOINT": {
    "2026-01-01.2026-12-31": 541550,
    "2027-01-01.2027-12-31": 553050,
    "2028-01-01.2028-12-31": 564200,
    "2029-01-01.2029-12-31": 575400,
    "2030-01-01.2030-12-31": 586750,
    "2031-01-01.2031-12-31": 598350,
    "2032-01-01.2032-12-31": 610200,
    "2033-01-01.2033-12-31": 622300,
    "2034-01-01.2034-12-31": 634650,
    "2035-01-01.2100-12-31": 647250
  },
  "gov.irs.income.bracket.thresholds.6.JOINT": {
    "2026-01-01.2026-12-31": 611750,
    "2027-01-01.2027-12-31": 624700,
    "2028-01-01.2028-12-31": 637350,
    "2029-01-01.2029-12-31": 649950,
    "2030-01-01.2030-12-31": 662800,
    "2031-01-01.2031-12-31": 675900,
    "2032-01-01.2032-12-31": 689250,
    "2033-01-01.2033-12-31": 702950,
    "2034-01-01.2034-12-31": 716900,
    "2035-01-01.2100-12-31": 731150
  },
  "gov.irs.credits.ctc.amount.adult_dependent": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.senior_deduction.amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.income.amt.exemption.amount.SINGLE": {
    "2026-01-01.2026-12-31": 70600,
    "2027-01-01.2027-12-31": 72100,
    "2028-01-01.2028-12-31": 73500,
    "2029-01-01.2029-12-31": 75000,
    "2030-01-01.2030-12-31": 76400,
    "2031-01-01.2031-12-31": 78000,
    "2032-01-01.2032-12-31": 79500,
    "2033-01-01.2033-12-31": 81100,
    "2034-01-01.2034-12-31": 82700,
    "2035-01-01.2100-12-31": 84300
  },
  "gov.irs.income.bracket.thresholds.1.SINGLE": {
    "2026-01-01.2026-12-31": 12150,
    "2027-01-01.2027-12-31": 12400,
    "2028-01-01.2028-12-31": 12650,
    "2029-01-01.2029-12-31": 12900,
    "2030-01-01.2030-12-31": 13150,
    "2031-01-01.2031-12-31": 13425,
    "2032-01-01.2032-12-31": 13675,
    "2033-01-01.2033-12-31": 13950,
    "2034-01-01.2034-12-31": 14225,
    "2035-01-01.2100-12-31": 14500
  },
  "gov.irs.income.bracket.thresholds.2.SINGLE": {
    "2026-01-01.2026-12-31": 49300,
    "2027-01-01.2027-12-31": 50350,
    "2028-01-01.2028-12-31": 51400,
    "2029-01-01.2029-12-31": 52400,
    "2030-01-01.2030-12-31": 53450,
    "2031-01-01.2031-12-31": 54500,
    "2032-01-01.2032-12-31": 55550,
    "2033-01-01.2033-12-31": 56650,
    "2034-01-01.2034-12-31": 57800,
    "2035-01-01.2100-12-31": 58950
  },
  "gov.irs.income.bracket.thresholds.3.SINGLE": {
    "2026-01-01.2026-12-31": 119400,
    "2027-01-01.2027-12-31": 121950,
    "2028-01-01.2028-12-31": 124400,
    "2029-01-01.2029-12-31": 126900,
    "2030-01-01.2030-12-31": 129400,
    "2031-01-01.2031-12-31": 131950,
    "2032-01-01.2032-12-31": 134550,
    "2033-01-01.2033-12-31": 137200,
    "2034-01-01.2034-12-31": 139950,
    "2035-01-01.2100-12-31": 142750
  },
  "gov.irs.income.bracket.thresholds.4.SINGLE": {
    "2026-01-01.2026-12-31": 249100,
    "2027-01-01.2027-12-31": 254400,
    "2028-01-01.2028-12-31": 259550,
    "2029-01-01.2029-12-31": 264650,
    "2030-01-01.2030-12-31": 269900,
    "2031-01-01.2031-12-31": 275250,
    "2032-01-01.2032-12-31": 280700,
    "2033-01-01.2033-12-31": 286250,
    "2034-01-01.2034-12-31": 291900,
    "2035-01-01.2100-12-31": 297750
  },
  "gov.irs.income.bracket.thresholds.5.SINGLE": {
    "2026-01-01.2026-12-31": 541550,
    "2027-01-01.2027-12-31": 553050,
    "2028-01-01.2028-12-31": 564200,
    "2029-01-01.2029-12-31": 575400,
    "2030-01-01.2030-12-31": 586750,
    "2031-01-01.2031-12-31": 598350,
    "2032-01-01.2032-12-31": 610200,
    "2033-01-01.2033-12-31": 622300,
    "2034-01-01.2034-12-31": 634650,
    "2035-01-01.2100-12-31": 647250
  },
  "gov.irs.income.bracket.thresholds.6.SINGLE": {
    "2026-01-01.2026-12-31": 543800,
    "2027-01-01.2027-12-31": 555300,
    "2028-01-01.2028-12-31": 566500,
    "2029-01-01.2029-12-31": 577700,
    "2030-01-01.2030-12-31": 589150,
    "2031-01-01.2031-12-31": 600800,
    "2032-01-01.2032-12-31": 612700,
    "2033-01-01.2033-12-31": 624850,
    "2034-01-01.2034-12-31": 637250,
    "2035-01-01.2100-12-31": 649900
  },
  "gov.irs.deductions.itemized.casualty.active": {
    "2026-01-01.2100-12-31": True
  },
  "gov.irs.deductions.standard.amount.SEPARATE": {
    "2025-01-01.2025-12-31": 15000,
    "2026-01-01.2026-12-31": 8300,
    "2027-01-01.2027-12-31": 8450,
    "2028-01-01.2028-12-31": 8650,
    "2029-01-01.2029-12-31": 8800,
    "2030-01-01.2030-12-31": 9000,
    "2031-01-01.2031-12-31": 9150,
    "2032-01-01.2032-12-31": 9350,
    "2033-01-01.2033-12-31": 9500,
    "2034-01-01.2034-12-31": 9700,
    "2035-01-01.2100-12-31": 9900
  },
  "gov.irs.income.amt.exemption.phase_out.rate": {
    "2026-01-01.2100-12-31": 0.25
  },
  "gov.irs.deductions.overtime_income.cap.JOINT": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.qbi.max.w2_wages.alt_rate": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.income.amt.exemption.amount.SEPARATE": {
    "2026-01-01.2026-12-31": 54900,
    "2027-01-01.2027-12-31": 56050,
    "2028-01-01.2028-12-31": 57200,
    "2029-01-01.2029-12-31": 58350,
    "2030-01-01.2030-12-31": 59500,
    "2031-01-01.2031-12-31": 60650,
    "2032-01-01.2032-12-31": 61850,
    "2033-01-01.2033-12-31": 63100,
    "2034-01-01.2034-12-31": 64350,
    "2035-01-01.2100-12-31": 65600
  },
  "gov.irs.income.bracket.thresholds.1.SEPARATE": {
    "2026-01-01.2026-12-31": 12150,
    "2027-01-01.2027-12-31": 12400,
    "2028-01-01.2028-12-31": 12650,
    "2029-01-01.2029-12-31": 12900,
    "2030-01-01.2030-12-31": 13150,
    "2031-01-01.2031-12-31": 13425,
    "2032-01-01.2032-12-31": 13675,
    "2033-01-01.2033-12-31": 13950,
    "2034-01-01.2034-12-31": 14225,
    "2035-01-01.2100-12-31": 14500
  },
  "gov.irs.income.bracket.thresholds.2.SEPARATE": {
    "2026-01-01.2026-12-31": 49300,
    "2027-01-01.2027-12-31": 50350,
    "2028-01-01.2028-12-31": 51400,
    "2029-01-01.2029-12-31": 52400,
    "2030-01-01.2030-12-31": 53450,
    "2031-01-01.2031-12-31": 54500,
    "2032-01-01.2032-12-31": 55550,
    "2033-01-01.2033-12-31": 56650,
    "2034-01-01.2034-12-31": 57800,
    "2035-01-01.2100-12-31": 58950
  },
  "gov.irs.income.bracket.thresholds.3.SEPARATE": {
    "2026-01-01.2026-12-31": 99500,
    "2027-01-01.2027-12-31": 101625,
    "2028-01-01.2028-12-31": 103675,
    "2029-01-01.2029-12-31": 105725,
    "2030-01-01.2030-12-31": 107800,
    "2031-01-01.2031-12-31": 109950,
    "2032-01-01.2032-12-31": 112125,
    "2033-01-01.2033-12-31": 114350,
    "2034-01-01.2034-12-31": 116600,
    "2035-01-01.2100-12-31": 118925
  },
  "gov.irs.income.bracket.thresholds.4.SEPARATE": {
    "2026-01-01.2026-12-31": 151625,
    "2027-01-01.2027-12-31": 154850,
    "2028-01-01.2028-12-31": 157975,
    "2029-01-01.2029-12-31": 161100,
    "2030-01-01.2030-12-31": 164275,
    "2031-01-01.2031-12-31": 167525,
    "2032-01-01.2032-12-31": 170850,
    "2033-01-01.2033-12-31": 174225,
    "2034-01-01.2034-12-31": 177700,
    "2035-01-01.2100-12-31": 181225
  },
  "gov.irs.income.bracket.thresholds.5.SEPARATE": {
    "2026-01-01.2026-12-31": 270775,
    "2027-01-01.2027-12-31": 276525,
    "2028-01-01.2028-12-31": 282100,
    "2029-01-01.2029-12-31": 287700,
    "2030-01-01.2030-12-31": 293375,
    "2031-01-01.2031-12-31": 299175,
    "2032-01-01.2032-12-31": 305100,
    "2033-01-01.2033-12-31": 311150,
    "2034-01-01.2034-12-31": 317325,
    "2035-01-01.2100-12-31": 323625
  },
  "gov.irs.income.bracket.thresholds.6.SEPARATE": {
    "2026-01-01.2026-12-31": 305875,
    "2027-01-01.2027-12-31": 312350,
    "2028-01-01.2028-12-31": 318675,
    "2029-01-01.2029-12-31": 324975,
    "2030-01-01.2030-12-31": 331400,
    "2031-01-01.2031-12-31": 337950,
    "2032-01-01.2032-12-31": 344625,
    "2033-01-01.2033-12-31": 351475,
    "2034-01-01.2034-12-31": 358450,
    "2035-01-01.2100-12-31": 365575
  },
  "gov.irs.credits.ctc.phase_out.threshold.JOINT": {
    "2026-01-01.2100-12-31": 110000
  },
  "gov.irs.credits.ctc.refundable.individual_max": {
    "2025-01-01.2025-12-31": 1800,
    "2026-01-01.2100-12-31": 1000
  },
  "gov.irs.deductions.overtime_income.cap.SINGLE": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.phase_out.threshold.SINGLE": {
    "2026-01-01.2100-12-31": 75000
  },
  "gov.irs.deductions.itemized.charity.ceiling.all": {
    "2026-01-01.2100-12-31": 0.5
  },
  "gov.irs.deductions.overtime_income.cap.SEPARATE": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.phase_out.threshold.SEPARATE": {
    "2026-01-01.2100-12-31": 55000
  },
  "gov.irs.credits.ctc.adult_ssn_requirement_applies": {
    "2025-01-01.2100-12-31": False
  },
  "gov.irs.credits.ctc.refundable.phase_in.threshold": {
    "2026-01-01.2100-12-31": 3000
  },
  "gov.irs.deductions.itemized.charity.floor.applies": {
    "2026-01-01.2100-12-31": False
  },
  "gov.irs.deductions.qbi.max.business_property.rate": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.income.amt.exemption.phase_out.start.JOINT": {
    "2026-01-01.2026-12-31": 209200,
    "2027-01-01.2027-12-31": 213600,
    "2028-01-01.2028-12-31": 217900,
    "2029-01-01.2029-12-31": 222200,
    "2030-01-01.2030-12-31": 226600,
    "2031-01-01.2031-12-31": 231100,
    "2032-01-01.2032-12-31": 235700,
    "2033-01-01.2033-12-31": 240300,
    "2034-01-01.2034-12-31": 245100,
    "2035-01-01.2100-12-31": 250000
  },
  "gov.irs.deductions.standard.amount.SURVIVING_SPOUSE": {
    "2025-01-01.2025-12-31": 30000,
    "2026-01-01.2026-12-31": 16600,
    "2027-01-01.2027-12-31": 16900,
    "2028-01-01.2028-12-31": 17300,
    "2029-01-01.2029-12-31": 17600,
    "2030-01-01.2030-12-31": 18000,
    "2031-01-01.2031-12-31": 18300,
    "2032-01-01.2032-12-31": 18700,
    "2033-01-01.2033-12-31": 19000,
    "2034-01-01.2034-12-31": 19400,
    "2035-01-01.2100-12-31": 19800
  },
  "gov.irs.income.amt.exemption.phase_out.start.SINGLE": {
    "2026-01-01.2026-12-31": 156900,
    "2027-01-01.2027-12-31": 160200,
    "2028-01-01.2028-12-31": 163400,
    "2029-01-01.2029-12-31": 166700,
    "2030-01-01.2030-12-31": 170000,
    "2031-01-01.2031-12-31": 173300,
    "2032-01-01.2032-12-31": 176800,
    "2033-01-01.2033-12-31": 180300,
    "2034-01-01.2034-12-31": 183800,
    "2035-01-01.2100-12-31": 187500
  },
  "gov.irs.deductions.standard.amount.HEAD_OF_HOUSEHOLD": {
    "2025-01-01.2025-12-31": 22500,
    "2026-01-01.2026-12-31": 12150,
    "2027-01-01.2027-12-31": 12400,
    "2028-01-01.2028-12-31": 12650,
    "2029-01-01.2029-12-31": 12900,
    "2030-01-01.2030-12-31": 13200,
    "2031-01-01.2031-12-31": 13450,
    "2032-01-01.2032-12-31": 13700,
    "2033-01-01.2033-12-31": 14000,
    "2034-01-01.2034-12-31": 14250,
    "2035-01-01.2100-12-31": 14550
  },
  "gov.irs.income.amt.exemption.amount.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 109800,
    "2027-01-01.2027-12-31": 112100,
    "2028-01-01.2028-12-31": 114400,
    "2029-01-01.2029-12-31": 116700,
    "2030-01-01.2030-12-31": 119000,
    "2031-01-01.2031-12-31": 121300,
    "2032-01-01.2032-12-31": 123700,
    "2033-01-01.2033-12-31": 126200,
    "2034-01-01.2034-12-31": 128700,
    "2035-01-01.2100-12-31": 131200
  },
  "gov.irs.income.bracket.thresholds.1.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 24300,
    "2027-01-01.2027-12-31": 24800,
    "2028-01-01.2028-12-31": 25300,
    "2029-01-01.2029-12-31": 25800,
    "2030-01-01.2030-12-31": 26300,
    "2031-01-01.2031-12-31": 26850,
    "2032-01-01.2032-12-31": 27350,
    "2033-01-01.2033-12-31": 27900,
    "2034-01-01.2034-12-31": 28450,
    "2035-01-01.2100-12-31": 29000
  },
  "gov.irs.income.bracket.thresholds.2.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 98600,
    "2027-01-01.2027-12-31": 100700,
    "2028-01-01.2028-12-31": 102800,
    "2029-01-01.2029-12-31": 104800,
    "2030-01-01.2030-12-31": 106900,
    "2031-01-01.2031-12-31": 109000,
    "2032-01-01.2032-12-31": 111100,
    "2033-01-01.2033-12-31": 113300,
    "2034-01-01.2034-12-31": 115600,
    "2035-01-01.2100-12-31": 117900
  },
  "gov.irs.income.bracket.thresholds.3.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 199000,
    "2027-01-01.2027-12-31": 203250,
    "2028-01-01.2028-12-31": 207350,
    "2029-01-01.2029-12-31": 211450,
    "2030-01-01.2030-12-31": 215600,
    "2031-01-01.2031-12-31": 219900,
    "2032-01-01.2032-12-31": 224250,
    "2033-01-01.2033-12-31": 228700,
    "2034-01-01.2034-12-31": 233200,
    "2035-01-01.2100-12-31": 237850
  },
  "gov.irs.income.bracket.thresholds.4.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 303250,
    "2027-01-01.2027-12-31": 309700,
    "2028-01-01.2028-12-31": 315950,
    "2029-01-01.2029-12-31": 322200,
    "2030-01-01.2030-12-31": 328550,
    "2031-01-01.2031-12-31": 335050,
    "2032-01-01.2032-12-31": 341700,
    "2033-01-01.2033-12-31": 348450,
    "2034-01-01.2034-12-31": 355400,
    "2035-01-01.2100-12-31": 362450
  },
  "gov.irs.income.bracket.thresholds.5.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 541550,
    "2027-01-01.2027-12-31": 553050,
    "2028-01-01.2028-12-31": 564200,
    "2029-01-01.2029-12-31": 575400,
    "2030-01-01.2030-12-31": 586750,
    "2031-01-01.2031-12-31": 598350,
    "2032-01-01.2032-12-31": 610200,
    "2033-01-01.2033-12-31": 622300,
    "2034-01-01.2034-12-31": 634650,
    "2035-01-01.2100-12-31": 647250
  },
  "gov.irs.income.bracket.thresholds.6.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 611750,
    "2027-01-01.2027-12-31": 624700,
    "2028-01-01.2028-12-31": 637350,
    "2029-01-01.2029-12-31": 649950,
    "2030-01-01.2030-12-31": 662800,
    "2031-01-01.2031-12-31": 675900,
    "2032-01-01.2032-12-31": 689250,
    "2033-01-01.2033-12-31": 702950,
    "2034-01-01.2034-12-31": 716900,
    "2035-01-01.2100-12-31": 731150
  },
  "gov.irs.income.amt.exemption.amount.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 70600,
    "2027-01-01.2027-12-31": 72100,
    "2028-01-01.2028-12-31": 73500,
    "2029-01-01.2029-12-31": 75000,
    "2030-01-01.2030-12-31": 76400,
    "2031-01-01.2031-12-31": 78000,
    "2032-01-01.2032-12-31": 79500,
    "2033-01-01.2033-12-31": 81100,
    "2034-01-01.2034-12-31": 82700,
    "2035-01-01.2100-12-31": 84300
  },
  "gov.irs.income.amt.exemption.phase_out.start.SEPARATE": {
    "2026-01-01.2026-12-31": 104600,
    "2027-01-01.2027-12-31": 106800,
    "2028-01-01.2028-12-31": 108950,
    "2029-01-01.2029-12-31": 111100,
    "2030-01-01.2030-12-31": 113300,
    "2031-01-01.2031-12-31": 115550,
    "2032-01-01.2032-12-31": 117850,
    "2033-01-01.2033-12-31": 120150,
    "2034-01-01.2034-12-31": 122550,
    "2035-01-01.2100-12-31": 125000
  },
  "gov.irs.income.bracket.thresholds.1.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 17350,
    "2027-01-01.2027-12-31": 17700,
    "2028-01-01.2028-12-31": 18050,
    "2029-01-01.2029-12-31": 18400,
    "2030-01-01.2030-12-31": 18800,
    "2031-01-01.2031-12-31": 19150,
    "2032-01-01.2032-12-31": 19550,
    "2033-01-01.2033-12-31": 19900,
    "2034-01-01.2034-12-31": 20300,
    "2035-01-01.2100-12-31": 20700
  },
  "gov.irs.income.bracket.thresholds.2.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 66050,
    "2027-01-01.2027-12-31": 67450,
    "2028-01-01.2028-12-31": 68850,
    "2029-01-01.2029-12-31": 70200,
    "2030-01-01.2030-12-31": 71550,
    "2031-01-01.2031-12-31": 73000,
    "2032-01-01.2032-12-31": 74450,
    "2033-01-01.2033-12-31": 75900,
    "2034-01-01.2034-12-31": 77400,
    "2035-01-01.2100-12-31": 78950
  },
  "gov.irs.income.bracket.thresholds.3.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 170550,
    "2027-01-01.2027-12-31": 174150,
    "2028-01-01.2028-12-31": 177700,
    "2029-01-01.2029-12-31": 181200,
    "2030-01-01.2030-12-31": 184800,
    "2031-01-01.2031-12-31": 188450,
    "2032-01-01.2032-12-31": 192150,
    "2033-01-01.2033-12-31": 195950,
    "2034-01-01.2034-12-31": 199850,
    "2035-01-01.2100-12-31": 203850
  },
  "gov.irs.income.bracket.thresholds.4.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 276200,
    "2027-01-01.2027-12-31": 282050,
    "2028-01-01.2028-12-31": 287750,
    "2029-01-01.2029-12-31": 293450,
    "2030-01-01.2030-12-31": 299250,
    "2031-01-01.2031-12-31": 305150,
    "2032-01-01.2032-12-31": 311200,
    "2033-01-01.2033-12-31": 317350,
    "2034-01-01.2034-12-31": 323650,
    "2035-01-01.2100-12-31": 330100
  },
  "gov.irs.income.bracket.thresholds.5.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 541550,
    "2027-01-01.2027-12-31": 553050,
    "2028-01-01.2028-12-31": 564200,
    "2029-01-01.2029-12-31": 575400,
    "2030-01-01.2030-12-31": 586750,
    "2031-01-01.2031-12-31": 598350,
    "2032-01-01.2032-12-31": 610200,
    "2033-01-01.2033-12-31": 622300,
    "2034-01-01.2034-12-31": 634650,
    "2035-01-01.2100-12-31": 647250
  },
  "gov.irs.income.bracket.thresholds.6.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 577750,
    "2027-01-01.2027-12-31": 590000,
    "2028-01-01.2028-12-31": 601950,
    "2029-01-01.2029-12-31": 613850,
    "2030-01-01.2030-12-31": 625950,
    "2031-01-01.2031-12-31": 638350,
    "2032-01-01.2032-12-31": 651000,
    "2033-01-01.2033-12-31": 663900,
    "2034-01-01.2034-12-31": 677050,
    "2035-01-01.2100-12-31": 690500
  },
  "gov.irs.deductions.itemized.interest.mortgage.cap.JOINT": {
    "2026-01-01.2100-12-31": 1000000
  },
  "gov.irs.deductions.overtime_income.cap.SURVIVING_SPOUSE": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.qbi.deduction_floor.amount[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.cdcc.phase_out.amended_structure.applies": {
    "2026-01-01.2100-12-31": False
  },
  "gov.irs.credits.ctc.phase_out.threshold.SURVIVING_SPOUSE": {
    "2026-01-01.2100-12-31": 75000
  },
  "gov.irs.deductions.itemized.interest.mortgage.cap.SINGLE": {
    "2026-01-01.2100-12-31": 1000000
  },
  "gov.irs.deductions.overtime_income.cap.HEAD_OF_HOUSEHOLD": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.phase_out.threshold.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2100-12-31": 75000
  },
  "gov.irs.deductions.itemized.interest.mortgage.cap.SEPARATE": {
    "2026-01-01.2100-12-31": 500000
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.cap.JOINT": {
    "2025-01-01.2025-12-31": 10000,
    "2026-01-01.2100-12-31": 1000000000000
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.cap.SINGLE": {
    "2025-01-01.2025-12-31": 10000,
    "2026-01-01.2100-12-31": 1000000000000
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.cap.SEPARATE": {
    "2025-01-01.2025-12-31": 5000,
    "2026-01-01.2100-12-31": 1000000000000
  },
  "gov.irs.income.amt.exemption.phase_out.start.SURVIVING_SPOUSE": {
    "2026-01-01.2026-12-31": 209200,
    "2027-01-01.2027-12-31": 213600,
    "2028-01-01.2028-12-31": 217900,
    "2029-01-01.2029-12-31": 222200,
    "2030-01-01.2030-12-31": 226600,
    "2031-01-01.2031-12-31": 231100,
    "2032-01-01.2032-12-31": 235700,
    "2033-01-01.2033-12-31": 240300,
    "2034-01-01.2034-12-31": 245100,
    "2035-01-01.2100-12-31": 250000
  },
  "gov.irs.deductions.itemized.charity.non_itemizers_amount.JOINT": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.income.amt.exemption.phase_out.start.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2026-12-31": 156900,
    "2027-01-01.2027-12-31": 160200,
    "2028-01-01.2028-12-31": 163400,
    "2029-01-01.2029-12-31": 166700,
    "2030-01-01.2030-12-31": 170000,
    "2031-01-01.2031-12-31": 173300,
    "2032-01-01.2032-12-31": 176800,
    "2033-01-01.2033-12-31": 180300,
    "2034-01-01.2034-12-31": 183800,
    "2035-01-01.2100-12-31": 187500
  },
  "gov.irs.deductions.itemized.charity.non_itemizers_amount.SINGLE": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.itemized.reduction.amended_structure.applies": {
    "2026-01-01.2100-12-31": False
  },
  "gov.irs.deductions.itemized.charity.non_itemizers_amount.SEPARATE": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.itemized.interest.mortgage.cap.SURVIVING_SPOUSE": {
    "2026-01-01.2100-12-31": 1000000
  },
  "gov.irs.deductions.itemized.interest.mortgage.cap.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2100-12-31": 1000000
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.phase_out.in_effect": {
    "2025-01-01.2029-12-31": False
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.cap.SURVIVING_SPOUSE": {
    "2025-01-01.2025-12-31": 10000,
    "2026-01-01.2100-12-31": 1000000000000
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.cap.HEAD_OF_HOUSEHOLD": {
    "2025-01-01.2025-12-31": 10000,
    "2026-01-01.2100-12-31": 1000000000000
  },
  "gov.irs.deductions.itemized.salt_and_real_estate.phase_out.floor.applies": {
    "2025-01-01.2029-12-31": False
  },
  "gov.irs.deductions.itemized.charity.non_itemizers_amount.SURVIVING_SPOUSE": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.irs.deductions.itemized.charity.non_itemizers_amount.HEAD_OF_HOUSEHOLD": {
    "2026-01-01.2100-12-31": 0
  }
}, country_id="us")


reformed = Microsimulation(reform=reform, dataset = "hf://policyengine/test/sparse_cd_stacked_2023.h5")

In [14]:
# Apply the same state_fips correction to the reformed simulation
cd_geoids_reform = reformed.calculate("congressional_district_geoid").values
correct_state_fips_reform = cd_geoids_reform // 100
reformed.set_input("state_fips", 2023, correct_state_fips_reform)

# Delete any cached calculations to force recalculation
if "state_name" in reformed.tax_benefit_system.variables:
    reformed.delete_arrays("state_name", 2023)
if "state_code" in reformed.tax_benefit_system.variables:
    reformed.delete_arrays("state_code", 2023)

In [15]:
income_tax_reform = reformed.calculate("income_tax", map_to="household", period=2026)


In [17]:
household_net_income_reform = reformed.calculate("household_net_income", map_to="household", period=2026)

KeyboardInterrupt: 